У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [86]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer, StandardScaler, RobustScaler, PolynomialFeatures, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, roc_curve, auc, root_mean_squared_error, r2_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from operator import pos

from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [ ]:
customer_df = pd.read_csv("/content/customer_segmentation_train.csv")
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [82]:
customer_df.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [ ]:
customer_df.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [ ]:
for col in customer_df.select_dtypes(include='object').columns:
    print(f"Column: {col}")
    value_counts = customer_df[col].value_counts(dropna=False, normalize=True) * 100  # Враховуємо NaN
    print(value_counts)
    print(f"Пропущені значення: {customer_df[col].isna().sum()} ({customer_df[col].isna().mean() * 100:.2f}%)")
    print("-" * 50)

Column: Gender
Gender
Male      54.747149
Female    45.252851
Name: proportion, dtype: float64
Пропущені значення: 0 (0.00%)
--------------------------------------------------
Column: Ever_Married
Ever_Married
Yes    57.548339
No     40.716411
NaN     1.735250
Name: proportion, dtype: float64
Пропущені значення: 140 (1.74%)
--------------------------------------------------
Column: Graduated
Graduated
Yes    61.576599
No     37.456619
NaN     0.966782
Name: proportion, dtype: float64
Пропущені значення: 78 (0.97%)
--------------------------------------------------
Column: Profession
Profession
Artist           31.184928
Healthcare       16.509668
Entertainment    11.762519
Engineer          8.663857
Doctor            8.527516
Lawyer            7.721864
Executive         7.424393
Marketing         3.619236
Homemaker         3.049083
NaN               1.536936
Name: proportion, dtype: float64
Пропущені значення: 124 (1.54%)
--------------------------------------------------
Column: Spend

Бачимо, що пропущених значень дуже мало, в середньому до 1 відсотка від усіх даних.
Стратегії обробки пустих значень та кодування -
1. Column: Var_1 - для пропущених значень створимо додаткову категорію cat_8
2. Column: Spending_Score - Оскільки значення мають певний порядок (Low < Average < High), логічно використати Ordinal Encoding
3. Column: Profession - для пропущених значень використаємо нову категорію undefined
4. Column: Ever_Married
   Column: Graduated
   Пропущені значення заповнимо домінуючим значенням

In [133]:
numeric_cols

['Age', 'Work_Experience', 'Family_Size']

In [134]:
train_df, val_df = train_test_split(customer_df, test_size=0.2, random_state=42, stratify=customer_df.Segmentation)

# Створюємо трен. і вал. набори
input_cols = list(train_df.columns)[1:-1]
target_col = 'Segmentation'
train_inputs, train_targets = train_df[input_cols], train_df[target_col]
val_inputs, val_targets = val_df[input_cols], val_df[target_col]


binary_cols = ['Ever_Married', 'Graduated', 'Gender']
spending_col = ['Spending_Score']
nominal_cols = ['Profession', 'Gender', 'Var_1']
numeric_cols = ['Work_Experience']
numeric_int_cols = ['Age', 'Family_Size']

# ['Ever_Married', 'Graduated', 'Gender']
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

# ['Spending_Score']
spending_order = [['Low', 'Average', 'High']]
spending_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=spending_order))
])

# ['Profession', 'Var_1']
nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('encoder', OneHotEncoder(sparse_output=False))
])

# ['Age', 'Work_Experience', 'Family_Size']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
])

# ['Age', 'Family_Size']
numeric_int_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('binary', binary_transformer, binary_cols),
    ('nominal', nominal_transformer, nominal_cols),
    ('spending', spending_transformer, spending_col),
    ('numeric', numeric_transformer, numeric_cols),
    ('numeric_int', numeric_int_transformer, numeric_int_cols),
])


# data for smotenc
mean_imputer = SimpleImputer(strategy='mean')
frequent_imputer = SimpleImputer(strategy='most_frequent')
constant_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

train_inputs_smotenc = train_inputs.copy()

train_inputs_smotenc[numeric_cols + numeric_int_cols] = mean_imputer.fit_transform(train_inputs_smotenc[numeric_cols + numeric_int_cols])

train_inputs_smotenc[spending_col + binary_cols] = frequent_imputer.fit_transform(train_inputs_smotenc[spending_col + binary_cols])

train_inputs_smotenc[nominal_cols] = constant_imputer.fit_transform(train_inputs_smotenc[nominal_cols])


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [79]:
train_inputs_transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   constatnt_imputer __Profession     6454 non-null   object 
 1   constatnt_imputer __Gender         6454 non-null   object 
 2   constatnt_imputer __Var_1          6454 non-null   object 
 3   frequent_imputer __Spending_Score  6454 non-null   object 
 4   frequent_imputer __Ever_Married    6454 non-null   object 
 5   frequent_imputer __Graduated       6454 non-null   object 
 6   frequent_imputer __Gender          6454 non-null   object 
 7   mean_imputer__Age                  6454 non-null   float64
 8   mean_imputer__Work_Experience      6454 non-null   float64
 9   mean_imputer__Family_Size          6454 non-null   float64
dtypes: float64(3), object(7)
memory usage: 504.3+ KB


In [118]:
? SMOTENC

In [131]:
X_train_smotenc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7256 entries, 0 to 7255
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Gender           7256 non-null   category
 1   Ever_Married     7256 non-null   category
 2   Age              7256 non-null   float64 
 3   Graduated        7256 non-null   category
 4   Profession       7256 non-null   category
 5   Work_Experience  7256 non-null   float64 
 6   Spending_Score   7256 non-null   category
 7   Family_Size      7256 non-null   float64 
 8   Var_1            7256 non-null   category
dtypes: category(6), float64(3)
memory usage: 213.9 KB


In [153]:
# SMOTENC
categorical_trans_cols = train_inputs_smotenc.select_dtypes(include=['object']).columns.tolist()
train_inputs_smotenc[categorical_trans_cols] = train_inputs_smotenc[categorical_trans_cols].astype('category')

smotenc = SMOTENC(categorical_features='auto', random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(train_inputs_smotenc, train_targets)
X_train_smotenc_df = pd.DataFrame(X_train_smotenc, columns=train_inputs_smotenc.columns.tolist())

print("Розподіл класів до SMOTENC:\n", train_targets.value_counts(normalize=True))
print("Розподіл класів після SMOTENC:\n", pd.Series(y_train_smotenc).value_counts(normalize=True))


Розподіл класів до SMOTENC:
 Segmentation
D    0.281066
A    0.244500
C    0.244190
B    0.230245
Name: proportion, dtype: float64
Розподіл класів після SMOTENC:
 Segmentation
A    0.25
B    0.25
C    0.25
D    0.25
Name: proportion, dtype: float64


In [136]:
# SMOTE-Tomek
train_inputs_full_transformed = preprocessor.fit_transform(train_inputs)
transformed_full_feature_names = preprocessor.get_feature_names_out()
train_inputs_full_transformed_df = pd.DataFrame(train_inputs_full_transformed, columns=transformed_full_feature_names)

smotetomek = SMOTETomek(random_state=0)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(train_inputs_full_transformed_df, train_targets)

In [137]:
# SMOTE
smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(train_inputs_full_transformed_df, train_targets)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [124]:
X_train_smotenc_df.describe()

,Age,Work_Experience,Family_Size
count,7256.000000,7256.000000,7256.000000
mean,43.950591,2.655376,2.814777
std,16.532877,3.243671,1.472603
min,18.000000,0.000000,1.000000
25%,31.000000,0.000000,2.000000
50%,41.000000,1.000000,2.839742
75%,53.000000,4.000000,4.000000
max,89.000000,14.000000,9.000000


In [155]:
logreg = OneVsRestClassifier(LogisticRegression(solver='liblinear', max_iter=10000))

pipeline = Pipeline(steps=[('classifier', logreg)])
pipeline.fit(X_train_smote, y_train_smote)

val_inputs_transformed = preprocessor.transform(val_inputs)
val_inputs_transformed_df = pd.DataFrame(val_inputs_transformed, columns=transformed_full_feature_names)

y_pred_smote = pipeline.predict(val_inputs_transformed_df)

print("Classification Report for SMOTE:")
print(classification_report(val_targets, y_pred_smote))

pipeline.fit(X_train_smotetomek, y_train_smotetomek)
y_pred_smotetomek = pipeline.predict(val_inputs_transformed_df)

print("Classification Report for SMOTETOMEK:")
print(classification_report(val_targets, y_pred_smotetomek))


pipeline_smotenk = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', logreg)])

pipeline_smotenk.fit(X_train_smotenc_df, y_train_smotenc)
y_pred_smotenk= pipeline_smotenk.predict(val_inputs)

print("Classification Report for SMOTENC:")
print(classification_report(val_targets, y_pred_smotetomek))


pipeline_smotenk.fit(train_inputs, train_targets)
y_pred_raw= pipeline_smotenk.predict(val_inputs)

print("Classification Report for inbalanced data:")
print(classification_report(val_targets, y_pred_raw))

Classification Report for SMOTE:
              precision    recall  f1-score   support

           A       0.42      0.48      0.45       394
           B       0.42      0.22      0.28       372
           C       0.50      0.61      0.55       394
           D       0.67      0.71      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.49      1614
weighted avg       0.51      0.52      0.50      1614

Classification Report for SMOTETOMEK:
              precision    recall  f1-score   support

           A       0.42      0.50      0.46       394
           B       0.41      0.21      0.28       372
           C       0.49      0.61      0.54       394
           D       0.68      0.71      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.49      1614
weighted avg       0.51      0.52      0.50      1614

Classification Report for SMOTENC:
              precision 

1. За основну метрику обрано macro avg для того, щоб оцінити модель на усіх класах надаючи їх однакову важливість.
2. Для вирішення проблеми диспабалну застосували 3 різні методи оверсамплінгу. Бачимо, що всі моделі показали однакові результати, окрім моделі, де ми не балансували класи (вона показала трохи гірші результати).
D    28.111056
A    24.442241
C    24.417452
B    23.029251

бачимо, що з самого початку класи доволі добре збалансовані, найбільше відхилення становить 5%, тому дял балансування не довелося створювати велику кількість нових ʼштучнихʼ значень, через це моделі могли показати однакові результати.
Найкраще моделі визначають найбільш представлений клас D, а найгірше - найменш представлений клас В.